In [ ]:
!pip install --upgrade --quiet  qdrant-client langchain

In [ ]:
# USING QDRANT CLIENT

import json
from qdrant_client import QdrantClient


def create_qdrant_collection():
    # Load data from JSON file
    with open("data/test_1k_rows.json") as f:
        data = json.load(f)

    # Extract text and vectors from data
    texts = [item["response_value"] for item in data]
    vectors = [item["embeddings"] for item in data]

    # Initialize Qdrant client in local mode
    client = QdrantClient()

    # Create collection
    collection_name = "test_collection_1k_rows"
    client.create_collection(collection_name=collection_name)

    # Insert documents into collection
    for i, text in enumerate(texts):
        vector = vectors[i]
        document = {"text": text, "vector": vector}
        client.insert(collection_name, [document])

    print(f"Qdrant collection '{collection_name}' created successfully.")


# Call the function to create the Qdrant collection
create_qdrant_collection()

In [ ]:
# USING LANGCHAIN AND ASYNC

import asyncio
from langchain_community.vectorstores import Qdrant


def create_qdrant_collection():
    # Load data from JSON file
    with open("data/test_1k_rows.json") as f:
        data = json.load(f)

    # Extract text and vectors from data
    texts = [item["response_value"] for item in data]
    vectors = [item["embeddings"] for item in data]
    type = [item["type"] for item in data]
    created = [item["created"] for item in data]
    subject_page_path = [item["subject_page_path"] for item in data]
    feedack_record_id = [item["feedack_record_id"] for item in data]
    response_identifier = [item["response_identifier"] for item in data]

    docs = []

    for i, text in enumerate(texts):
        document = {
            "text": text,
            "vector": vectors[i],
            "metadata": {
                "type": type[i],
                "created": created[i],
                "subject_page_path": subject_page_path[i],
                "feedack_record_id": feedack_record_id[i],
                "response_identifier": response_identifier[i],
            },
        }
        docs.append(document)

    async def process_document(document):
        try:
            await Qdrant.from_documents(
                documents=document,
                url="localhost:6334",
                prefer_grpc=True,
                collection_name="test_async_collection_1k_rows",
                force_recreate=False,
            )
        except Exception as e:
            print(f"Error processing document: {e}")

        # create a list of tasks
        tasks = [process_document(doc) for doc in docs]

        # run all tasks concurrently
        await asyncio.gather(*tasks)

    print(f"Qdrant collection 'my_collection' created successfully.")